In [1]:
!pip install matplotlib

In [2]:
import gurobipy as gp
from gurobipy import GRB
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader  # Corrected import
from torch_geometric.nn import MessagePassing
from torch_geometric.nn import global_mean_pool  # For pooling in the decoder
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import random
import matplotlib.pyplot as plt  # For plotting losses

In [3]:
from parse_QP import my_callback

In [4]:
# Read the problem
number = "0031"
grb_model = gp.read(f"QPLIB_{number}.lp")

# Solution storage
grb_model._feasible_solutions = []
grb_model._relaxation_solutions = []
grb_model.setParam("MIPGap", 0.05)
#model.setParam("NodeLimit", 100)  # Explore a limited number of nodes

# Optimize
grb_model.optimize(my_callback)

Restricted license - for non-production use only - expires 2026-11-23
Read LP format model from file QPLIB_0031.lp
Reading time = 0.00 seconds
obj: 32 rows, 60 columns, 120 nonzeros
Set parameter MIPGap to value 0.05
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 22.04.5 LTS")

CPU model: AMD EPYC 7513 32-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 64 physical cores, 128 logical processors, using up to 32 threads

Non-default parameters:
MIPGap  0.05

Optimize a model with 32 rows, 60 columns and 120 nonzeros
Model fingerprint: 0x00d24133
Model has 464 quadratic objective terms
Variable types: 30 continuous, 30 integer (30 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [3e+01, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Found heuristic solution: objective 3654.4800000
Presolve time: 0.00s
Presolved: 901 rows, 526 columns, 2321 nonzeros
Presolved 

In [5]:
# Retrieve optimal solution if available
if grb_model.status == GRB.OPTIMAL:
    optimal_solution = grb_model.getAttr('X', grb_model.getVars())
    print("Optimal solution:", optimal_solution)
else:
    print(f"Model status: {grb_model.status}")

Optimal solution: [0.0, 0.0, 0.49055482068657474, 0.0, 0.0, 0.09207045008492966, 0.0, 0.0, 0.06558348815658188, 0.2063472207852159, 0.0, 0.14544402028669787, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0, -0.0, 1.0, -0.0, -0.0, 1.0, 0.0, 0.0, 1.0, 1.0, -0.0, 1.0, -0.0, 0.0, -0.0, 0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0]


In [6]:
## Extracting bounds

import gurobipy as gp

# Read the problem
number = "0031"
grb_model = gp.read(f"QPLIB_{number}.lp")

variable_bounds = {}
for var in grb_model.getVars():
    variable_bounds[var.VarName] = {'Lower': var.LB, 'Upper': var.UB}


Read LP format model from file QPLIB_0031.lp
Reading time = 0.00 seconds
obj: 32 rows, 60 columns, 120 nonzeros


In [7]:
## Extracting Q, A, b, d, etc

from parse_QP import parse_qplib_file

# Replace '0031' with the desired file number
data = parse_qplib_file('0031')

# Access the data
A = data['A']
b_vector = data['b_vector']
E = data['E']
d = data['d']
Q = data['Q']
variables_info = data['variables_info']
variables_info = [v[0] for v in variables_info]
binary_indices = data['binary_indices']
variable_indices = data['variable_indices']

m, n = A.shape
# Get indices of non-zero elements in A
row_indices, col_indices = np.nonzero(A)
edge_weights = A[row_indices, col_indices]

# Map variable types to numerical values
# Node types: 0 - continuous, 1 - binary
variable_types = np.array([0 if v[0] == 'x' else 1 for v in variables_info])

# Collect indices of continuous and binary variables
continuous_indices = np.where(variable_types == 0)[0]
binary_indices = np.where(variable_types == 1)[0]
n_continuous = len(continuous_indices)
n_binary = len(binary_indices)


In [8]:
### Generating data and just reloading it if it's already generated (to save time)

import os
import pickle
from generate_solutions import generate_feasible_solutions, generate_infeasible_solutions

# Set generate_new to False by default
generate_new = False
feasible_data_file = 'feasible_data.pkl'
infeasible_data_file = 'infeasible_data.pkl'

if not generate_new and os.path.exists(feasible_data_file) and os.path.exists(infeasible_data_file):
    # Load feasible data
    with open(feasible_data_file, 'rb') as f:
        feasible_data = pickle.load(f)
    feasible_solutions = feasible_data['solutions']
    feasible_costs = feasible_data['costs']

    # Load infeasible data
    with open(infeasible_data_file, 'rb') as f:
        infeasible_data = pickle.load(f)
    infeasible_solutions = infeasible_data['solutions']
    infeasible_costs = infeasible_data['costs']

    print("Loaded existing feasible and infeasible solutions from files.")
else:
    # Generate feasible solutions
    num_objectives = 20000  # Adjust the number as needed
    feasible_solutions, feasible_costs = generate_feasible_solutions(A, E, Q, variables_info, b_vector, d, num_objectives)

    # Generate infeasible solutions
    num_infeasible_samples = len(feasible_solutions) * 1  # For balance
    infeasible_solutions, infeasible_costs = generate_infeasible_solutions(
        A=A,
        E=E,
        variables_info=variables_info,
        b_vector=b_vector,
        d=d,
        Q=Q,
        num_infeasible_samples=num_infeasible_samples,
        feasible_solutions=feasible_solutions
    )

    print(f"Total unique solutions collected for both feasible/infeasible: {len(feasible_solutions) + len(infeasible_solutions)}")

    # Save feasible data
    with open(feasible_data_file, 'wb') as f:
        pickle.dump({'solutions': feasible_solutions, 'costs': feasible_costs}, f)

    # Save infeasible data
    with open(infeasible_data_file, 'wb') as f:
        pickle.dump({'solutions': infeasible_solutions, 'costs': infeasible_costs}, f)

    print("Generated new feasible and infeasible solutions and saved to files.")


Loaded existing feasible and infeasible solutions from files.


In [9]:
## Creating the dataset (graph representation)

data_list = []

# Number of edge types
num_edge_types = 3  # 0: ineq, 1: eq, 2: var-var

def prepare_samples(x_samples, costs):
    """
    Prepares samples by computing constraint violations and creating Data objects.

    Args:
        x_samples (list of np.ndarray): List of variable assignments.
        costs (list of float): List of corresponding cost values.

    Returns:
        list of Data objects: Prepared data samples.
    """
    samples = []
    for x_combined, cost in tqdm(zip(x_samples, costs), total=len(x_samples)):
        # Prepare node features and types
        variable_node_features = x_combined.reshape(-1, 1)
        variable_node_type = variable_types  # 0: continuous, 1: binary

        # Prepare constraint node features and types
        inequality_node_features = b_vector.reshape(-1, 1)
        inequality_node_type = np.full(m, 2)  # Inequality constraint node type

        equality_node_features = d.reshape(-1, 1)
        equality_node_type = np.full(len(d), 3)  # Equality constraint node type

        # Concatenate all node features and types
        node_features = np.concatenate([variable_node_features,
                                        inequality_node_features,
                                        equality_node_features], axis=0)

        node_types = np.concatenate([variable_node_type,
                                     inequality_node_type,
                                     equality_node_type], axis=0)

        # Edge indices and attributes for inequality constraints
        ineq_row_indices, ineq_col_indices = np.nonzero(A)
        ineq_edge_weights = A[ineq_row_indices, ineq_col_indices]
        ineq_source_nodes = ineq_col_indices
        ineq_target_nodes = ineq_row_indices + n  # Shift indices for inequality nodes
        ineq_edge_index = np.stack([ineq_source_nodes, ineq_target_nodes], axis=0)
        ineq_edge_attr = ineq_edge_weights.reshape(-1, 1)

        # Edge indices and attributes for equality constraints
        eq_row_indices, eq_col_indices = np.nonzero(E)
        eq_edge_weights = E[eq_row_indices, eq_col_indices]
        eq_source_nodes = eq_col_indices
        eq_target_nodes = eq_row_indices + n + m  # Shift indices for equality nodes
        eq_edge_index = np.stack([eq_source_nodes, eq_target_nodes], axis=0)
        eq_edge_attr = eq_edge_weights.reshape(-1, 1)

        # Edge indices and attributes for variable-variable edges (from Q)
        q_row_indices, q_col_indices = np.nonzero(Q)
        q_edge_weights = Q[q_row_indices, q_col_indices]

        # Include both directions since edges are directed
        var_var_source_nodes = np.concatenate([q_row_indices, q_col_indices])
        var_var_target_nodes = np.concatenate([q_col_indices, q_row_indices])
        var_var_edge_index = np.stack([var_var_source_nodes,
                                       var_var_target_nodes], axis=0)
        var_var_edge_attr = np.concatenate([q_edge_weights, q_edge_weights]).reshape(-1, 1)

        # Edge types
        ineq_edge_type = np.full(len(ineq_edge_attr), 0)
        eq_edge_type = np.full(len(eq_edge_attr), 1)
        var_var_edge_type = np.full(len(var_var_edge_attr), 2)

        # Combine edge indices, attributes, and types
        edge_index = np.concatenate([ineq_edge_index,
                                     eq_edge_index,
                                     var_var_edge_index], axis=1)
        edge_attr = np.concatenate([ineq_edge_attr,
                                    eq_edge_attr,
                                    var_var_edge_attr], axis=0)
        edge_type = np.concatenate([ineq_edge_type,
                                    eq_edge_type,
                                    var_var_edge_type], axis=0)

        # One-hot encode edge types
        edge_type_onehot = np.zeros((len(edge_type), num_edge_types))
        edge_type_onehot[np.arange(len(edge_type)), edge_type] = 1

        # Concatenate edge attributes and edge type one-hot encoding
        edge_attr = np.concatenate([edge_attr, edge_type_onehot], axis=1)

        # Convert to torch tensors
        x = torch.tensor(node_features, dtype=torch.float)
        node_type_tensor = torch.tensor(node_types, dtype=torch.long)
        edge_index_torch = torch.tensor(edge_index, dtype=torch.long)
        edge_attr_torch = torch.tensor(edge_attr, dtype=torch.float)
        cost_tensor = torch.tensor([cost], dtype=torch.float)

        # Compute inequality constraint violations: A x - b
        Ax_minus_b = A @ x_combined - b_vector  # Shape: (m,)
        inequality_violations = torch.tensor(Ax_minus_b, dtype=torch.float)

        # Compute equality constraint violations: E x - d
        Ex_minus_d = E @ x_combined - d  # Shape: (p,)
        equality_violations = torch.tensor(Ex_minus_d, dtype=torch.float)

        # Create Data object
        data = Data(
            x=x,
            edge_index=edge_index_torch,
            edge_attr=edge_attr_torch,
            inequality_violations=inequality_violations,
            equality_violations=equality_violations,
            node_type=node_type_tensor,
            cost=cost_tensor
        )
        samples.append(data)
    return samples
    
print("Preparing feasible samples...")
feasible_samples = prepare_samples(
    feasible_solutions, feasible_costs)

print("Preparing infeasible samples...")
infeasible_samples = prepare_samples(
    infeasible_solutions, infeasible_costs)

# Combine samples
data_list = feasible_samples + infeasible_samples

# Shuffle the dataset
random.shuffle(data_list)

# Split Dataset into Training and Validation
train_ratio = 0.8
train_size = int(len(data_list) * train_ratio)
train_data_list = data_list[:train_size]
val_data_list = data_list[train_size:]

print(f"Training samples: {len(train_data_list)}")
print(f"Validation samples: {len(val_data_list)}")

# Create DataLoaders
from torch_geometric.loader import DataLoader

batch_size = 64  # Adjust based on your memory capacity
train_loader = DataLoader(train_data_list, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data_list, batch_size=batch_size, shuffle=False)

Preparing feasible samples...


100%|██████████| 9297/9297 [00:02<00:00, 3272.90it/s]


Preparing infeasible samples...


100%|██████████| 9297/9297 [00:02<00:00, 3230.17it/s]

Training samples: 14875
Validation samples: 3719


In [13]:
from networks import Decoder, InequalityViolationPredictor, EqualityViolationPredictor, CostPredictor

class ConstraintGNN(MessagePassing):
    def __init__(self, input_node_feature_size, edge_feature_size, hidden_size):
        super(ConstraintGNN, self).__init__(aggr='mean')  # Mean aggregation
        self.message_mlp = nn.Sequential(
            nn.Linear(input_node_feature_size + edge_feature_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size)
        )
        self.update_mlp = nn.Sequential(
            nn.Linear(input_node_feature_size + hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size)
        )

    def forward(self, x, edge_index, edge_attr):
        return self.propagate(edge_index, x=x, edge_attr=edge_attr)

    def message(self, x_j, edge_attr):
        m_input = torch.cat([x_j, edge_attr], dim=1)
        m = self.message_mlp(m_input)
        return m

    def update(self, aggr_out, x):
        u = torch.cat([x, aggr_out], dim=1)
        u = self.update_mlp(u)
        return u

class GCNEncoder(nn.Module):
    def __init__(self, node_feature_size, edge_feature_size,
                 hidden_size, latent_size, num_layers):
        super(GCNEncoder, self).__init__()
        self.layers = nn.ModuleList()
        self.layers.append(ConstraintGNN(node_feature_size,
                                         edge_feature_size, hidden_size))
        for _ in range(num_layers - 1):
            self.layers.append(ConstraintGNN(hidden_size,
                                             edge_feature_size, hidden_size))
        self.global_pool = global_mean_pool
        self.fc_mu = nn.Linear(hidden_size, latent_size)
        self.fc_logvar = nn.Linear(hidden_size, latent_size)

    def forward(self, x, edge_index, edge_attr, batch):
        for layer in self.layers:
            x = layer(x, edge_index, edge_attr)
        # Global pooling
        x = self.global_pool(x, batch)
        # Compute mean and log variance for latent variables
        mu = self.fc_mu(x)
        logvar = self.fc_logvar(x)
        return mu, logvar

class VAEModel(nn.Module):
    def __init__(self, node_feature_size, edge_feature_size,
                 hidden_size, latent_size, num_layers,
                 continuous_size, binary_size,
                 num_inequality_constraints, num_equality_constraints):
        super(VAEModel, self).__init__()
        self.encoder = GCNEncoder(node_feature_size, edge_feature_size,
                                  hidden_size, latent_size, num_layers)
        self.decoder = Decoder(latent_size, hidden_size,
                               continuous_size, binary_size)
        self.inequality_violation_predictor = InequalityViolationPredictor(
            latent_size, num_inequality_constraints)
        self.equality_violation_predictor = EqualityViolationPredictor(
            latent_size, num_equality_constraints)
        self.cost_predictor = CostPredictor(latent_size)
        self.continuous_size = continuous_size
        self.binary_size = binary_size
        self.n = continuous_size + binary_size

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, data):
        x, edge_index, edge_attr, node_type, batch = (
            data.x, data.edge_index, data.edge_attr,
            data.node_type, data.batch
        )
        mu, logvar = self.encoder(x, edge_index, edge_attr, batch)
        z = self.reparameterize(mu, logvar)
        x_recon = self.decoder(z)
        inequality_violation_pred = self.inequality_violation_predictor(z)
        equality_violation_pred = self.equality_violation_predictor(z)
        c_pred = self.cost_predictor(z)
        return x_recon, mu, logvar, inequality_violation_pred, equality_violation_pred, c_pred


# ==============================
# 9. Instantiate the Model
# ==============================

# Number of edge types (as defined in dataset creation)
num_edge_types = 3
node_feature_size = 1  # One feature per node
edge_feature_size = 1 + num_edge_types  # Edge weight + edge type one-hot
hidden_size = 64 
latent_size = 32  
num_layers = 3  

# Sizes for continuous and binary variables
continuous_size = n_continuous  # Number of continuous variables
binary_size = n_binary          # Number of binary variables

# Number of constraints
num_inequality_constraints = A.shape[0]  # m
num_equality_constraints = E.shape[0]    # p

model = VAEModel(
    node_feature_size, edge_feature_size, hidden_size, latent_size, num_layers,
    continuous_size, binary_size, num_inequality_constraints, num_equality_constraints
)

# Optionally move the model to GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Precompute indices tensors for variable types
model.continuous_indices = torch.tensor(continuous_indices, dtype=torch.long).to(device)
model.binary_indices = torch.tensor(binary_indices, dtype=torch.long).to(device)

# ==============================
# 5. Define Optimizer and Loss Functions
# ==============================

optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

# Loss weights
beta = 1.0      # Weight for KL divergence loss
gamma = 1.0     # Weight for inequality violation loss
gamma_eq = 1.0  # Weight for equality violation loss
delta = 0.1   # Weight for cost prediction loss

# Loss functions
reconstruction_loss_fn_continuous = nn.MSELoss()
reconstruction_loss_fn_binary = nn.BCELoss()
violation_loss_fn = nn.MSELoss() 
cost_loss_fn = nn.L1Loss()

# ==============================
# 6. Training the VAE
# ==============================

num_epochs = 10  # Adjust the number of epochs as needed

print("Starting training...")
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    for batch_idx, batch in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")):
        batch = batch.to(device)
        optimizer.zero_grad()
        # Forward pass
        x_recon, mu, logvar, inequality_violation_pred, equality_violation_pred, c_pred = model(batch)
        x = batch.x
        node_type = batch.node_type
        batch_size = batch.num_graphs

        # Reshape x and node_type
        x = x.view(batch_size, -1, 1)
        node_type = node_type.view(batch_size, -1)

        # Extract variable nodes
        x_variable = x[:, :model.n, :].squeeze(-1)
        node_type_variable = node_type[:, :model.n]

        # Split reconstructed variables
        x_recon_continuous = x_recon[:, :model.continuous_size]
        x_recon_binary = x_recon[:, model.continuous_size:]

        # Extract ground truth variables
        x_variable_continuous = x_variable[:, continuous_indices]
        x_variable_binary = x_variable[:, binary_indices]

        # Reconstruction Loss
        recon_loss_continuous = reconstruction_loss_fn_continuous(x_recon_continuous, x_variable_continuous)
        recon_loss_binary = reconstruction_loss_fn_binary(x_recon_binary, x_variable_binary)
        recon_loss = recon_loss_continuous + recon_loss_binary

        # KL Divergence Loss
        kl_loss = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())

        # **Reshape constraint violations to match predictions**
        actual_inequality_violations = batch.inequality_violations.view(batch_size, -1).to(device)
        actual_equality_violations = batch.equality_violations.view(batch_size, -1).to(device)

        # Inequality Violation Loss
        inequality_violation_loss = violation_loss_fn(inequality_violation_pred, actual_inequality_violations)

        # Equality Violation Loss
        equality_violation_loss = violation_loss_fn(equality_violation_pred, actual_equality_violations)

        # Cost Prediction Loss
        c = batch.cost.to(device)
        cost_loss = cost_loss_fn(c_pred.view(-1), c)

        # Total Loss
        loss = (
            recon_loss +
            beta * kl_loss +
            gamma * inequality_violation_loss +
            gamma_eq * equality_violation_loss +
            delta * cost_loss
        )
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        total_loss += loss.item()

        # Diagnostic Logging
        if batch_idx % 100 == 0:
            print(f"Batch {batch_idx+1}, Recon Loss: {recon_loss.item():.4f}, KL Loss: {kl_loss.item():.4f}, "
                  f"Ineq Violation Loss: {inequality_violation_loss.item():.4f}, "
                  f"Eq Violation Loss: {equality_violation_loss.item():.4f}, "
                  f"Cost Loss: {cost_loss.item():.4f}")

Starting training...


Epoch 1/10:   3%|▎         | 7/233 [00:00<00:03, 66.88it/s]

Batch 1, Recon Loss: 1.4498, KL Loss: 8.0630, Ineq Violation Loss: 1.0317, Eq Violation Loss: 1.6623, Cost Loss: 7126.6025


Epoch 1/10:  48%|████▊     | 111/233 [00:01<00:01, 70.18it/s]

Batch 101, Recon Loss: 0.3074, KL Loss: 59.0498, Ineq Violation Loss: 0.0338, Eq Violation Loss: 0.3193, Cost Loss: 5774.2998


...

Batch 101, Recon Loss: 0.2765, KL Loss: 1.5898, Ineq Violation Loss: 0.0317, Eq Violation Loss: 0.2690, Cost Loss: 4728.2324


Epoch 10/10:  90%|█████████ | 210/233 [00:03<00:00, 61.74it/s]

Batch 201, Recon Loss: 0.2725, KL Loss: 1.2980, Ineq Violation Loss: 0.0279, Eq Violation Loss: 0.3290, Cost Loss: 4478.4795


Epoch 10/10: 100%|██████████| 233/233 [00:03<00:00, 64.59it/s]


Epoch 10/10, Training Loss: 476.440666
Training completed.
